In [62]:
%pip install scikit-learn matplotlib
%pip install grecy 
%run -m grecy install grc_perseus_trf

/Users/pletcher/.pyenv/versions/3.12.7/lib/python3.12/pty.py:95: RuntimeWarning: Using fork() can cause Polars to deadlock in the child process.
In addition, using fork() with Python in general is a recipe for mysterious
deadlocks and crashes.

The most likely reason you are seeing this error is because you are using the
multiprocessing module on Linux, which uses fork() by default. This will be
fixed in Python 3.14. Until then, you want to use the "spawn" context instead.

See https://docs.pola.rs/user-guide/misc/multiprocessing/ for details.

  pid, fd = os.forkpty()



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-11-28_tragedy-dfs/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-11-28_tragedy-dfs/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

Installing grc_perseus_trf.....

Please wait, this could take some minutes.....

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/497.3 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.1/497.3 MB 3.7 MB/s eta 0:02:15
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.4/497.3 MB 4.5 MB/s eta 0:01:52
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/497.3 MB 9.9 MB/s eta 0:00:51
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/497.3 MB 14.9 MB/s eta 0:00:34
     ━━━

In [1]:
import json

import polars as pl

df = pl.read_parquet("./tragedy.parquet")
with open("./messenger_speeches_updated_no-satyr-play.json") as f:
    plays = json.load(f)

messenger_pre_df = []

for play in plays:
    dramatist = play["dramatist"]
    total_lines = play["total_lines"]
    total_messenger_lines = 0

    title = play["title"]

    for line_pair in play["speeches"]:
        total_messenger_lines += line_pair[1] - line_pair[0]

        for line in range(line_pair[0], line_pair[1] + 1):
            # print(dramatist, title, line)

            ref = df.filter(
                pl.col("dramatist") == dramatist,
                pl.col("title") == title,
                pl.col("n") == line,
            )

            if not ref["text"].is_empty():
                speaker = ref[0].select("speaker").item()

                if speaker != "Χορός":
                    line_obj = dict(
                        dramatist=dramatist,
                        title=title,
                        n=line,
                        speaker=speaker,
                        text=ref["text"].str.join(" ").item(),
                        total_lines=total_lines,
                        year=play["year"]
                    )

                    messenger_pre_df.append(line_obj)

        # print(f"{play['title']}: messenger_lines: {total_messenger_lines}; total_lines: {total_lines}; pct_messenger_lines: {(total_messenger_lines / total_lines) * 100}")

messenger_df = pl.DataFrame(messenger_pre_df)

In [16]:
by_play_and_speaker = messenger_df.group_by(
    pl.col("dramatist"), 
    pl.col("title"), 
    pl.col("speaker"),
    pl.col("n")
).agg(pl.col("text"), pl.col("total_lines").first(), pl.col("year").first())

by_play_and_speaker_grouped_text = messenger_df.group_by(
    pl.col("dramatist"), 
    pl.col("title"), 
    pl.col("speaker"),
).agg(pl.col("text"), pl.col("n").gather([0, -1]), pl.col("total_lines").first(), pl.col("year").first())

## Plotting where messenger speeches occur in tragedy

In [115]:
import altair as alt

base = alt.Chart(
    by_play_and_speaker.sort(pl.col("year"), pl.col("n")).with_columns(
        pl.concat_str(
            [pl.col("dramatist"), pl.col("title"), pl.col("speaker")], separator=" "
        ).alias("Dramatist-Title-Speaker"),
        percent=pl.col("n") / pl.col("total_lines"),
    )
).mark_point().encode(
    alt.Y("Dramatist-Title-Speaker", sort=alt.Sort(order="ascending", field="year")),
    alt.X("percent").title("Percentage through play"),
    color=alt.Color("title").title("Title"),
).properties(
    title="Occurrence of messenger speeches throughout Attic tragedy (by percentage of total lines)",
    height=1000,
    width="container", 
)

# Messengers become more reliable the later they appear in the play -- especially apparent for plays that have early vs. late messengers
# Trachiniae is an exception -- maybe?
base

alt.Chart(...)

In [114]:
alt.Chart(
    by_play_and_speaker.sort(pl.col("year"), pl.col("n")).with_columns(
        pl.concat_str(
            [pl.col("dramatist"), pl.col("title"), pl.col("speaker")], separator=" "
        ).alias("Dramatist-Title-Speaker"),
        percent=pl.col("n") / pl.col("total_lines")
    )
).transform_density("percent", as_=["Dramatist-Title-Speaker", "percent"]).mark_area().encode(
    alt.X("Dramatist-Title-Speaker:Q").title("Percentage through play"),
    alt.Y("percent").title("Density"),
).properties(
    title="Density of occurrences of messenger speeches throughout Attic tragedy (by percentage of total lines)",
    height=500,
    width="container"
)

alt.Chart(...)

In [37]:
# Authors: The scikit-learn developers
# SPDX-License-Identifier: BSD-3-Clause

from time import time

import matplotlib.pyplot as plt

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation, MiniBatchNMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20
batch_size = 128
init = "nndsvda"


def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(30, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[-n_top_words:]
        top_features = feature_names[top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f"Topic {topic_idx +1}", fontdict={"fontsize": 30})
        ax.tick_params(axis="both", which="major", labelsize=20)
        for i in "top right left".split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
import spacy

nlp = spacy.load('grc_perseus_trf')

In [66]:
data = by_play_and_speaker.select(pl.col('text').list.join(" "))
raw_samples = data['text'].to_list()

data_samples = []

STOPS = ["δέ", "τε", "ἀλλ", "ἀλλά", "οὔτε"]

with nlp.select_pipes(enable="lemmatizer"):
    for row in raw_samples:
        doc = nlp(row)
        lemmata = [t.lemma_ for t in doc]

        data_samples.append(' '.join([t.lemma_ for t in doc if not t.is_stop and t.lemma_ not in STOPS]))

In [ ]:

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(
    max_df=0.95, min_df=2, max_features=n_features, stop_words="english"
)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print(
    "Fitting the NMF model (Frobenius norm) with tf-idf features, "
    "n_samples=%d and n_features=%d..." % (n_samples, n_features)
)
t0 = time()
nmf = NMF(
    n_components=n_components,
    random_state=1,
    init=init,
    beta_loss="frobenius",
    alpha_W=0.00005,
    alpha_H=0.00005,
    l1_ratio=1,
).fit(tfidf)
print("done in %0.3fs." % (time() - t0))


tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
plot_top_words(
    nmf, tfidf_feature_names, n_top_words, "Topics in NMF model (Frobenius norm)"
)

# Fit the NMF model
print(
    "\n" * 2,
    "Fitting the NMF model (generalized Kullback-Leibler "
    "divergence) with tf-idf features, n_samples=%d and n_features=%d..."
    % (n_samples, n_features),
)
t0 = time()
nmf = NMF(
    n_components=n_components,
    random_state=1,
    init=init,
    beta_loss="kullback-leibler",
    solver="mu",
    max_iter=1000,
    alpha_W=0.00005,
    alpha_H=0.00005,
    l1_ratio=0.5,
).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
plot_top_words(
    nmf,
    tfidf_feature_names,
    n_top_words,
    "Topics in NMF model (generalized Kullback-Leibler divergence)",
)

# Fit the MiniBatchNMF model
print(
    "\n" * 2,
    "Fitting the MiniBatchNMF model (Frobenius norm) with tf-idf "
    "features, n_samples=%d and n_features=%d, batch_size=%d..."
    % (n_samples, n_features, batch_size),
)
t0 = time()
mbnmf = MiniBatchNMF(
    n_components=n_components,
    random_state=1,
    batch_size=batch_size,
    init=init,
    beta_loss="frobenius",
    alpha_W=0.00005,
    alpha_H=0.00005,
    l1_ratio=0.5,
).fit(tfidf)
print("done in %0.3fs." % (time() - t0))


tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
plot_top_words(
    mbnmf,
    tfidf_feature_names,
    n_top_words,
    "Topics in MiniBatchNMF model (Frobenius norm)",
)

# Fit the MiniBatchNMF model
print(
    "\n" * 2,
    "Fitting the MiniBatchNMF model (generalized Kullback-Leibler "
    "divergence) with tf-idf features, n_samples=%d and n_features=%d, "
    "batch_size=%d..." % (n_samples, n_features, batch_size),
)
t0 = time()
mbnmf = MiniBatchNMF(
    n_components=n_components,
    random_state=1,
    batch_size=batch_size,
    init=init,
    beta_loss="kullback-leibler",
    alpha_W=0.00005,
    alpha_H=0.00005,
    l1_ratio=0.5,
).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
plot_top_words(
    mbnmf,
    tfidf_feature_names,
    n_top_words,
    "Topics in MiniBatchNMF model (generalized Kullback-Leibler divergence)",
)

print(
    "\n" * 2,
    "Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
    % (n_samples, n_features),
)
lda = LatentDirichletAllocation(
    n_components=n_components,
    max_iter=5,
    learning_method="online",
    learning_offset=50.0,
    random_state=0,
)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

tf_feature_names = tf_vectorizer.get_feature_names_out()
plot_top_words(lda, tf_feature_names, n_top_words, "Topics in LDA model")